In [28]:
import pandas as pd
import numpy as np
import datetime 

### 读取数据

In [ ]:
User = pd.read_csv('./Data/JData_User.csv',encoding='GBK',na_values=-1,parse_dates=['user_reg_dt'])
User.head()

,user_id,age,sex,user_lv_cd,user_reg_dt
0,54,NaN,2,1,2016-01-26
1,79,36-45岁,2,2,2016-01-27
2,100,16-25岁,2,3,2016-01-27
3,266,15岁以下,2,2,2016-01-29
4,333,NaN,2,4,2016-01-30


In [ ]:
Action_201602 = pd.read_csv('./Data/JData_Action_201602.csv',\
                            encoding='GBK',\
                            na_values=-1,\
                            parse_dates=['time'],dtype={'user_id':'int','sku_id':'int','type':'int','model_id':'float64'})
Action_201602 = Action_201602.sort_values(['user_id','sku_id','time'])\
    .drop_duplicates(['user_id','sku_id','time'])

In [ ]:
Action_201603 = pd.read_csv('./Data/JData_Action_201603/JData_Action_201603.csv',\
                            encoding='GBK',\
                            na_values=-1,\
                            parse_dates=['time'],dtype={'user_id':'int','sku_id':'int','type':'int','model_id':'float64'})
Action_201603 = Action_201603.sort_values(['user_id','sku_id','time'])\
    .drop_duplicates(['user_id','sku_id','time'])

In [ ]:
Action_201603_extra = pd.read_csv('./Data/JData_Action_201603/JData_Action_201603_extra.csv',\
                            encoding='GBK',\
                            na_values=-1,\
                            parse_dates=['time'],dtype={'user_id':'int','sku_id':'int','type':'int','model_id':'float64'})
Action_201603_extra = Action_201603_extra.sort_values(['user_id','sku_id','time'])\
    .drop_duplicates(['user_id','sku_id','time'])

In [ ]:
Action_201603 = Action_201603.append(Action_201603_extra)

In [ ]:
Action_201604 = pd.read_csv('./Data/JData_Action_201604.csv',\
                            encoding='GBK',\
                            na_values=-1,\
                            parse_dates=['time'],dtype={'user_id':'int','sku_id':'int','type':'int','model_id':'float64'})

In [ ]:
Action_201604 = Action_201602.append(Action_201603).append(Action_201604)

In [ ]:
Action_201604.head()

In [ ]:
len(Action_201604)

In [ ]:
Action_201604 = Action_201604.sort_values(['user_id','sku_id','time'])\
    .drop_duplicates(['user_id','sku_id','time'])

In [ ]:
len(Action_201604)

In [ ]:
Action_201604 = Action_201604.merge(pd.get_dummies(Action_201604.type,prefix='type'),\
                                    left_index=True,
                                    right_index=True,
                                    how='left').drop(['cate','brand'],axis=1)
Action_201604.head()

In [ ]:
Action_201604 = Action_201604.groupby(['user_id','sku_id'])\
.agg({'type_1':sum,'type_2':sum,'type_3':sum,\
                    'type_4':sum,'type_5':sum,'type_6':sum,\
                    'time':min,'model_id':len
                   }).reset_index()

In [ ]:
Action_201604.head()

In [ ]:
Comments = pd.read_csv('./Data/JData_Comment.csv',encoding='GBK',na_values=-1,\
                       parse_dates=['dt'],dtype={'sku_id':'int'})
Comments.head()

In [ ]:
Product = pd.read_csv('./Data/JData_Product.csv',encoding='GBK',na_values=-1,\
                     dtype={'sku_id':'int'})
Product.head()

### 数据切分

In [ ]:
date0 = '2016-02-01'
date1 = '2016-04-06'
date2 = '2016-04-15'
date3 = '2016-04-16'

### 数据集合并

In [ ]:
def create_data(date_a,date_b):
    Action_section = Action_201604[(Action_201604['time'] >  date_a) & (Action_201604['time'] < date_b)]
    Comments_section =  Comments[Comments['dt'] < date_b]
    User_section =  User[User['user_reg_dt'] < date_b]
    User_section = User_section.sort_values('user_id')
#     Action_section = Action_section.sort_values(['user_id','sku_id','time']).drop_duplicates(['user_id','sku_id','time'])
    UA_section = Action_section.merge(User_section,on='user_id',how='left').sort_values(['user_id','sku_id','time'])
    Comments_section = Comments_section.sort_values(['sku_id','dt'],ascending=False).drop_duplicates(['sku_id'])
    PC_section = Product.merge(Comments_section,how='left',on='sku_id').sort_values('bad_comment_rate',ascending=False)
    PC_section = PC_section
    UAPC_section = PC_section.merge(UA_section,on='sku_id',how='left')
    UAPC_section = UAPC_section.dropna(axis=0,subset=['user_id'])
    UAPC_section[['user_id','sku_id']] = UAPC_section[['user_id','sku_id']].apply(lambda x:x.astype(int))
    return UAPC_section

In [ ]:
def create_data(date_a,date_b):
    Action_section = Action_201604[(Action_201604['time'] >  date_a) & (Action_201604['time'] < date_b)]
    Comments_section =  Comments[(Comments['dt'] >  date_a) & (Comments['dt'] < date_b)]
    User_section =  User[User['user_reg_dt'] < date_b]
    User_section = User_section.sort_values('user_id')
#     Action_section = Action_section.sort_values(['user_id','sku_id','time']).drop_duplicates(['user_id','sku_id','time'])
    UA_section = Action_section.merge(User_section,on='user_id',how='left').sort_values(['user_id','sku_id','time'])
    Comments_section = Comments_section.sort_values(['sku_id','dt'],ascending=False).drop_duplicates(['sku_id'])
    PC_section = Product.merge(Comments_section,how='left',on='sku_id').sort_values('bad_comment_rate',ascending=False)
    PC_section = PC_section
    UAPC_section = UA_section.merge(PC_section,on='sku_id',how='inner')
#     UAPC_section = UAPC_section.dropna(axis=0,subset=['user_id'])
    UAPC_section[['user_id','sku_id']] = UAPC_section[['user_id','sku_id']].apply(lambda x:x.astype(int))
    return UAPC_section

In [ ]:
def label_create(x):
    return x[['user_id','sku_id','type_4']]\
    .sort_values(['user_id','sku_id'])\
    .rename(columns= {'type_4':'label'})

In [ ]:
train_X = create_data(date0,date2)

In [ ]:
train_X['type_4'].value_counts()

In [ ]:
train_y = create_data(date2,date3)

In [ ]:
trial = train_y[train_y['type_4']>=1][['user_id','sku_id']].merge(Action_201604,how='left').sort_values(['user_id','sku_id'])

In [ ]:
trial.head()

In [ ]:
trial[type]

In [ ]:
Comments[Comments['sku_id']==5504]

In [ ]:
train_y.sort_values(['sku_id','time']).head()

In [ ]:
train_y['type_4'].value_counts()

In [ ]:
train_y = train_y[['user_id','sku_id','type_4']]\
    .sort_values(['user_id','sku_id'])\
    .rename(columns= {'type_4':'label'})

In [ ]:
train_y.head()

In [ ]:
train_test = train_X.merge(train_y,on=['user_id','sku_id'],how='left')

In [ ]:
train_test.head()

In [ ]:
train_test['label']= train_test['label'].fillna(0)

In [ ]:
train_test['label'].value_counts()